In [ ]:
import os
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Adding AWS S3 Minio configs
sparkConf = (
    SparkConf()
    .set("spark.jars.ivy","/home/brijeshdhaker/.ivy2")
    .set("spark.ui.port", "4042")
    #.set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    #.set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    #.set("spark.jars.packages","org.apache.hadoop:hadoop-aws:3.0.0,io.delta:delta-spark_2.12:3.3.2")
    #.set("spark.executor.heartbeatInterval", "300000")
    #.set("spark.network.timeout", "400000")
    #.set("spark.hadoop.fs.defaultFS", "s3a://defaultfs/")
    #.set("spark.hadoop.fs.s3a.endpoint", "http://minio.sandbox.net:9010")
    #.set("spark.hadoop.fs.s3a.access.key", "pgm2H2bR7a5kMc5XCYdO")
    #.set("spark.hadoop.fs.s3a.secret.key", "zjd8T0hXFGtfemVQ6AH3yBAPASJNXNbVSx5iddqG")
    #.set("spark.hadoop.fs.s3a.path.style.access", "true")
    #.set("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
    #.set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    #.set("spark.eventLog.enabled", "true")
    #.set("spark.eventLog.dir", "file:///apps/var/logs/spark-events")
)

spark = (
    SparkSession.builder.master("local[*]").
        appName('spark-joins-stratgies').
        config(conf=sparkConf).
        getOrCreate()
)

spark.sparkContext.setLogLevel('ERROR')
spark

25/10/10 10:50:07 WARN Utils: Your hostname, vmware-ubuntu-24.04 resolves to a loopback address: 127.0.1.1; using 192.168.154.133 instead (on interface ens33)
25/10/10 10:50:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/10 10:50:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Create a List Containing the "Column Names" for "Person"
personColumns = ["Id", "First_Name", "Last_Name", "AddressId"]

# Create a List Containing the "Data" for "Person"
personList = [\
                (1001, "Oindrila", "Chakraborty", "A001"),
                (1002, "Soumyajyoti", "Bagchi", "A002"),
                (1003, "Oishi", "Bhattacharyya", "A004"),
                (1004, "Sabarni", "Chakraborty", "A003"),
                (1005, "Ayan", "Dutta", "A002"),
                (1006, "Dhrubajyoti", "Das", "A004"),
                (1007, "Sayantan", "Chatterjee", "A004"),
                (1008, "Ritwik", "Ghosh", "A001"),
                (1009, "Puja", "Bhatt", "A001"),
                (1010, "Souvik", "Roy", "A002"),
                (1011, "Souvik", "Roy", "A003"),
                (1012, "Ria", "Ghosh", "A003"),
                (1013, "Soumyajit", "Pal", "A002"),
                (1014, "Abhirup", "Chakraborty", "A004"),
                (1015, "Sagarneel", "Sarkar", "A003"),
                (1016, "Anamika", "Pal", "A002"),
                (1017, "Swaralipi", "Roy", "A002"),
                (1018, "Rahul", "Roychowdhury", "A003"),
                (1019, "Paulomi", "Mondal", "A004"),
                (1020, "Avishek", "Basu", "A002"),
                (1021, "Avirupa", "Ghosh", "A004"),
                (1022, "Ameer", "Sengupta", "A003"),
                (1023, "Megha", "Kargupta", "A002"),
                (1024, "Madhura", "Chakraborty", "A002"),
                (1025, "Debankur", "Dutta", "A002"),
                (1026, "Bidisha", "Das", "A001"),
                (1027, "Rohan", "Ghosh", "A004"),
                (1028, "Tathagata", "Acharyya", "A003")
              ]

# Create a List Containing the "Column Names" for "Address"
addressColumns = ["AddressId", "Address"]

# Create a List Containing the "Data" for "Address"
addressList = [\
                  ("A001", "India"),
                  ("A002", "US"),
                  ("A003", "UK"),
                  ("A004", "UAE")
              ]

###
def display(df):
    df.show(truncate=False)


#### Create a DataFrame for “Person”.

In [3]:
dfPerson = spark.createDataFrame(personList, schema = personColumns)
dfPerson.printSchema()
display(dfPerson)

root
 |-- Id: long (nullable = true)
 |-- First_Name: string (nullable = true)
 |-- Last_Name: string (nullable = true)
 |-- AddressId: string (nullable = true)



+----+-----------+-------------+---------+
|Id  |First_Name |Last_Name    |AddressId|
+----+-----------+-------------+---------+
|1001|Oindrila   |Chakraborty  |A001     |
|1002|Soumyajyoti|Bagchi       |A002     |
|1003|Oishi      |Bhattacharyya|A004     |
|1004|Sabarni    |Chakraborty  |A003     |
|1005|Ayan       |Dutta        |A002     |
|1006|Dhrubajyoti|Das          |A004     |
|1007|Sayantan   |Chatterjee   |A004     |
|1008|Ritwik     |Ghosh        |A001     |
|1009|Puja       |Bhatt        |A001     |
|1010|Souvik     |Roy          |A002     |
|1011|Souvik     |Roy          |A003     |
|1012|Ria        |Ghosh        |A003     |
|1013|Soumyajit  |Pal          |A002     |
|1014|Abhirup    |Chakraborty  |A004     |
|1015|Sagarneel  |Sarkar       |A003     |
|1016|Anamika    |Pal          |A002     |
|1017|Swaralipi  |Roy          |A002     |
|1018|Rahul      |Roychowdhury |A003     |
|1019|Paulomi    |Mondal       |A004     |
|1020|Avishek    |Basu         |A002     |
+----+-----

#### Create Another DataFrame for “Address”.

In [4]:
dfAddress = spark.createDataFrame(addressList, schema = addressColumns)
dfAddress.printSchema()
display(dfAddress)

root
 |-- AddressId: string (nullable = true)
 |-- Address: string (nullable = true)

+---------+-------+
|AddressId|Address|
+---------+-------+
|A001     |India  |
|A002     |US     |
|A003     |UK     |
|A004     |UAE    |
+---------+-------+



#### 1. Shuffle Sort Merge Join

In [5]:
dfPersonWithAddress = dfPerson.join(dfAddress, dfPerson.AddressId == dfAddress.AddressId, "inner")
dfPersonWithAddress.printSchema()
display(dfPersonWithAddress)

root
 |-- Id: long (nullable = true)
 |-- First_Name: string (nullable = true)
 |-- Last_Name: string (nullable = true)
 |-- AddressId: string (nullable = true)
 |-- AddressId: string (nullable = true)
 |-- Address: string (nullable = true)

+----+-----------+------------+---------+---------+-------+
|Id  |First_Name |Last_Name   |AddressId|AddressId|Address|
+----+-----------+------------+---------+---------+-------+
|1001|Oindrila   |Chakraborty |A001     |A001     |India  |
|1008|Ritwik     |Ghosh       |A001     |A001     |India  |
|1009|Puja       |Bhatt       |A001     |A001     |India  |
|1026|Bidisha    |Das         |A001     |A001     |India  |
|1002|Soumyajyoti|Bagchi      |A002     |A002     |US     |
|1005|Ayan       |Dutta       |A002     |A002     |US     |
|1010|Souvik     |Roy         |A002     |A002     |US     |
|1013|Soumyajit  |Pal         |A002     |A002     |US     |
|1016|Anamika    |Pal         |A002     |A002     |US     |
|1017|Swaralipi  |Roy         |A002   

In [ ]:
dfPersonWithAddress.explain()

## In the “Physical Plan”, it can be seen that the “Join” performed is “Sort Merge Join” indeed.

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- SortMergeJoin [AddressId#3], [AddressId#25], Inner
   :- Sort [AddressId#3 ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(AddressId#3, 200), ENSURE_REQUIREMENTS, [plan_id=160]
   :     +- Filter isnotnull(AddressId#3)
   :        +- Scan ExistingRDD[Id#0L,First_Name#1,Last_Name#2,AddressId#3]
   +- Sort [AddressId#25 ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(AddressId#25, 200), ENSURE_REQUIREMENTS, [plan_id=161]
         +- Filter isnotnull(AddressId#25)
            +- Scan ExistingRDD[AddressId#25,Address#26]




#### “Disable” the “Sort Merge Join” as the “Default Join Selection Strategy”?

In [ ]:
spark.conf.set("spark.sql.join.preferSortMergeJoin", "false")

#### 2. Shuffle Hash Join

In [7]:
# Disable Broadcast Hash Join by setting the threshold to -1
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)
# “Disable” the “Sort Merge Join” as the “Default Join Selection Strategy”.
spark.conf.set("spark.sql.join.preferSortMergeJoin", "false")

In [8]:
dfPersonWithAddress = dfPerson.join(dfAddress.hint("SHUFFLE_HASH"), dfPerson.AddressId == dfAddress.AddressId, "inner")
dfPersonWithAddress.printSchema()
display(dfPersonWithAddress)

root
 |-- Id: long (nullable = true)
 |-- First_Name: string (nullable = true)
 |-- Last_Name: string (nullable = true)
 |-- AddressId: string (nullable = true)
 |-- AddressId: string (nullable = true)
 |-- Address: string (nullable = true)

+----+-----------+-------------+---------+---------+-------+
|Id  |First_Name |Last_Name    |AddressId|AddressId|Address|
+----+-----------+-------------+---------+---------+-------+
|1003|Oishi      |Bhattacharyya|A004     |A004     |UAE    |
|1002|Soumyajyoti|Bagchi       |A002     |A002     |US     |
|1001|Oindrila   |Chakraborty  |A001     |A001     |India  |
|1004|Sabarni    |Chakraborty  |A003     |A003     |UK     |
|1006|Dhrubajyoti|Das          |A004     |A004     |UAE    |
|1005|Ayan       |Dutta        |A002     |A002     |US     |
|1007|Sayantan   |Chatterjee   |A004     |A004     |UAE    |
|1008|Ritwik     |Ghosh        |A001     |A001     |India  |
|1009|Puja       |Bhatt        |A001     |A001     |India  |
|1011|Souvik     |Roy     

In [ ]:
dfPersonWithAddress.explain()

# In the “Physical Plan”, it can be seen that the “Join” performed is “Shuffle Hash Join” indeed.

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- ShuffledHashJoin [AddressId#3], [AddressId#25], Inner, BuildRight
   :- Exchange hashpartitioning(AddressId#3, 200), ENSURE_REQUIREMENTS, [plan_id=279]
   :  +- Filter isnotnull(AddressId#3)
   :     +- Scan ExistingRDD[Id#0L,First_Name#1,Last_Name#2,AddressId#3]
   +- Exchange hashpartitioning(AddressId#25, 200), ENSURE_REQUIREMENTS, [plan_id=280]
      +- Filter isnotnull(AddressId#25)
         +- Scan ExistingRDD[AddressId#25,Address#26]




#### 3. Broadcast Hash Join


In [13]:
# Enable Broadcast Hash Join by setting the threshold to -1
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 10485760)

In [ ]:
# 10 MB = 10 * 1024 * 1024 Bytes = 10485760 Bytes
print(spark.conf.get("spark.sql.autoBroadcastJoinThreshold"))

10485760


In [14]:
from pyspark.sql.functions import broadcast

dfPersonWithAddress = dfPerson.join(broadcast(dfAddress), dfPerson.AddressId == dfAddress.AddressId, "inner")
dfPersonWithAddress.printSchema()
display(dfPersonWithAddress)

root
 |-- Id: long (nullable = true)
 |-- First_Name: string (nullable = true)
 |-- Last_Name: string (nullable = true)
 |-- AddressId: string (nullable = true)
 |-- AddressId: string (nullable = true)
 |-- Address: string (nullable = true)

+----+-----------+-------------+---------+---------+-------+
|Id  |First_Name |Last_Name    |AddressId|AddressId|Address|
+----+-----------+-------------+---------+---------+-------+
|1001|Oindrila   |Chakraborty  |A001     |A001     |India  |
|1002|Soumyajyoti|Bagchi       |A002     |A002     |US     |
|1003|Oishi      |Bhattacharyya|A004     |A004     |UAE    |
|1004|Sabarni    |Chakraborty  |A003     |A003     |UK     |
|1005|Ayan       |Dutta        |A002     |A002     |US     |
|1006|Dhrubajyoti|Das          |A004     |A004     |UAE    |
|1007|Sayantan   |Chatterjee   |A004     |A004     |UAE    |
|1008|Ritwik     |Ghosh        |A001     |A001     |India  |
|1009|Puja       |Bhatt        |A001     |A001     |India  |
|1010|Souvik     |Roy     

In [ ]:
#
dfPersonWithAddress.explain()

# In the “Physical Plan”, it can be seen that the “Join” performed is “Broadcast Hash Join” indeed.

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastHashJoin [AddressId#3], [AddressId#25], Inner, BuildRight, false
   :- Filter isnotnull(AddressId#3)
   :  +- Scan ExistingRDD[Id#0L,First_Name#1,Last_Name#2,AddressId#3]
   +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, string, false]),false), [plan_id=365]
      +- Filter isnotnull(AddressId#25)
         +- Scan ExistingRDD[AddressId#25,Address#26]




#### 4. Iterative Broadcast Hash Join

In [ ]:
# First change the BrodcastJoinThreashold from 10MB to 100 MB
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 104857600)
print(spark.conf.get("spark.sql.autoBroadcastJoinThreshold"))

In [ ]:
# Step 1: Read the small table and persist in Memory
df_small = spark.read.parquet(path)
df_small_persist= df_small.persist()
df_small_persist.count()

#df_small.queryExecution.logical.optimizedPlan.stats.sizeInBytes

# Step 2: Get the size of the small table. Assume result gives 400MB
fst_txn_sz=df_small.queryExecution.logical.optimizedPlan.stats.sizeInBytes

# Step 3: Convert the No Of Chunk that the smaller table will be divide by dividing the size by BroadCastThreadshold Value which will give 4 as result.
no_of_chunk=(fst_txn_sz/(1024*1024*100)).toInt

# Step 4: Now divide the Smallest table into no_of_chuck by randomly assign the number to each record out of no of chuck. To acheive that we can use monotonically_increasing_id() function as shown below
df_small_broadcast=df_small.withColumn("chunk_id",monotonically_increasing_id()%num_of_pass)

# Step 5 : Iterate to Each broadcast and join with Largest table. Either you can union it or write it to tempory location then again join it. I prefer the write to temperory location.
for chunk in range(no_of_chunk):
  if(chunk%2==0):
    df_largest=spark.read.parquet(temp_path_even)
  else:
    df_largest=spark.read.parquet(temp_path_odd)

  df_larget.join(df_small_broadcast.where(chunk_id=chunk),column_condition,join)

  if(chunk&2==0 and chunk<no_of_chunk):
    spark.write.mode("overwrite").parquet(temp_path_odd)
  elif(chunk%2==1 and chunk<no_of_chunk):
    spark.write.mode("overwrite").parquet(temp_path_even)
  else:
    print("Done")

    


### Internal workings of Broadcast Nested Loop Join

#### There are 2 phases in a Broadcast Nested Loop Join.
Stages involved in a Broadcast Nested Loop Join
Broadcast Nested Loop join does not involve a shuffle or a sort. Smallest dataset of the two will be broadcasted to all partitions and a nested loop is performed between the 2 datasets to perform the join. Every record from dataset 1 is attempted to join with every record from dataset 2.

![spark stages](https://www.hadoopinrealworld.com/wp-content/uploads/2020/12/Broadcast-Nested-Loop-Join-Spark-stages.png)

#### Broadcast phase
Smallest dataset is broadcasted to all executors or tasks processing the bigger dataset
* Left side will be broadcasted in a right outer join.
* Right side in a left outer, left semi, left anti or existence join will be broadcasted.
* Either side can be broadcasted in an inner-like join.
![brodcase phase](https://www.hadoopinrealworld.com/wp-content/uploads/2020/12/Broadcast-Nested-Loop-Spark-stage-1.png)

#### Nested Loop Join phase
* Once the dataset is broadcasted, every record from one dataset is attempted to join with every record from another dataset in a nested loop.
* Since this join is used for non-equi conditions, the iteration can not stop as soon as a match is encountered like in Sort Merge Join. The iteration will go through the entire dataset.
* Note that a sort is not involved in this join.

![nested loop phase](https://www.hadoopinrealworld.com/wp-content/uploads/2020/12/Broadcast-Nested-Loop-Spark-stage-1-and-2.png)

In [ ]:
spark.conf.set("spark.sql.join.preferSortMergeJoin", True)
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 10485760)

In [ ]:
data1 = [10, 20, 20, 30, 40, 10, 40, 20, 20, 20, 20, 50]
data2 = [30, 20, 40, 50]

df1 = data1.toDF("id1")
df2 = data2.toDF("id2")

dfJoined = df1.join(df2, df1["id1"] >= df2["id2"])
dfJoined.foreach(lambda x : print(x))

In [ ]:
#
dfJoined.explain(extended=True)

In [ ]:
#
# Spark-3.0 support following 
#
dfJoined.explain(mode="simple")      # which will display the physical plan
dfJoined.explain(mode="extended")    # which will display physical and logical plans (like “extended” option)
dfJoined.explain(mode="codegen")     # which will display the java code planned to be executed
dfJoined.explain(mode="cost")        # which will display the optimized logical plan and related statistics (if they exist)
dfJoined.explain(mode="formatted")   # which will display a splitted output composed by a nice physical plan outline, and a section with each node details